In [1]:
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09b_rest.ipynb"
_nb_title = "Predictions using a REST endpoint"

### no need to change any of this
_nb_safeloc = _nb_loc.replace("/", "%2F")
md("""
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name={1}&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F{2}&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F{2}">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/{0}">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
""".format(_nb_loc, _nb_title, _nb_safeloc))


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Predictions using a REST endpoint&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F09_deploying%2F09b_rest.ipynb&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F09_deploying%2F09b_rest.ipynb">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09b_rest.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09b_rest.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/09_deploying/09b_rest.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


# Predictions using a REST endpoint

In this notebook, we start from an already trained and saved model (as in Chapter 7).
For convenience, we have put this model in a public bucket in gs://practical-ml-vision-book-data/flowers_5_trained

We deploy this model to a REST endpoint, and then show how to invoke the model using POST operations.

## REST endpoint

In [2]:
!cat ./vertex_deploy.sh

#!/bin/bash

REGION="us-central1"  # make sure you have GPU/TPU quota in this region
ENDPOINT_NAME="flowers_endpoint"
MODEL_NAME="flowers"
MODEL_LOCATION="gs://practical-ml-vision-book-data/flowers_5_trained"
IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-1:latest"

for i in "$@"
do
case $i in
        -r=*|--region=*) REGION="${i#*=}"; shift ;;
        -e=*|--endpoint_name=*) ENDPOINT_NAME="${i#*=}"; shift ;;
        -m=*|--model_name=*) MODEL_NAME="${i#*=}"; shift ;;
        -l=*|--model_location=*) MODEL_LOCATION="${i#*=}"; shift ;;
        -i=*|--image_uri=*) IMAGE_URI="${i#*=}"; shift ;;
        *) echo "Unknown parameter passed: $1"; exit 1 ;;
esac
done

echo "Deploying model $MODEL_NAME"

if [[ $(gcloud ai endpoints list --region=$REGION --format="value(display_name)" | grep $ENDPOINT_NAME) ]]; then
    echo "The endpoint named $ENDPOINT_NAME already exists."
else
    # Create endpoint.
    echo "Creating $ENDPOINT_NAME endpoint now."
    gcloud ai endpoints create \

In [3]:
!./vertex_deploy.sh

Deploying model flowers
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The endpoint named flowers_endpoint already exists.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The endpoint_id is 4327589805996113920
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Uploading flowers model now.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [8618855115064868864]...done.                            
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The model_id is 4626538221495386112
Deploying model now
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [3921600703717441536]...done.                            
Deployed a model to the endpoint 4327589805996113920. Id of the deployed model: 1963683786742824960.


## IMPORTANT: CHANGE THIS CELL

Note the endpoint ID and deployed model ID above. Set it in the cell below.

In [11]:
# CHANGE THESE TO REFLECT WHERE YOU DEPLOYED THE MODEL
import os
os.environ['ENDPOINT_ID'] = '4327589805996113920' # CHANGE
os.environ['MODEL_ID'] = '1963683786742824960' # CHANGE
os.environ['PROJECT'] = 'ai-analytics-solutions' # CHANGE
os.environ['BUCKET'] = 'ai-analytics-solutions-mlvisionbook' # CHANGE
os.environ['REGION'] = 'us-central1' # CHANGE

## JSON request

In [4]:
%%writefile request.json
{
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Writing request.json


In [12]:
%%bash
gcloud ai endpoints predict ${ENDPOINT_ID} \
--region=${REGION} \
--json-request=request.json \
--format=json

{
  "deployedModelId": "1963683786742824960",
  "model": "projects/379218021631/locations/us-central1/models/4626538221495386112",
  "modelDisplayName": "flowers",
  "predictions": [
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.619152546
    },
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.999984384
    },
    {
      "flower_type_int": 0,
      "flower_type_str": "daisy",
      "probability": 0.995082855
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.975185812
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.954917
    }
  ]
}


Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]


## Sending over HTTP Post

In [13]:
# Invoke from Python.
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
    REGION, PROJECT, REGION, ENDPOINT_ID)
headers = {"Authorization": "Bearer " + token }
data = {
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n  "predictions": [\n    {\n      "flower_type_int": 1,\n      "probability": 0.619152546,\n      "flower_type_str": "dandelion"\n    },\n    {\n      "flower_type_str": "dandelion",\n      "probability": 0.999984384,\n      "flower_type_int": 1\n    },\n    {\n      "probability": 0.995082855,\n      "flower_type_str": "daisy",\n      "flower_type_int": 0\n    },\n    {\n      "probability": 0.975185812,\n      "flower_type_str": "tulips",\n      "flower_type_int": 4\n    },\n    {\n      "flower_type_int": 4,\n      "flower_type_str": "tulips",\n      "probability": 0.954917\n    }\n  ],\n  "deployedModelId": "1963683786742824960",\n  "model": "projects/379218021631/locations/us-central1/models/4626538221495386112",\n  "modelDisplayName": "flowers"\n}\n'


## [Optional] CAIP Batch prediction

In [ ]:
%%writefile batchinputs.jsonl
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"}
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"}
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"}
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"}
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"}

In [ ]:
%%bash
gsutil cp batchinputs.jsonl gs://BUCKET

In [ ]:
# Invoke from Python.
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']
MODEL_ID = os.environ['MODEL_ID']
BUCKET = os.environ['BUCKET'] # used for staging

BATCH_JOB_NAME = "batch_pred_job"
INPUT_FORMAT = "jsonl"
INPUT_URI = "gs://{}/batchinputs.jsonl".format(BUCKET)
OUTPUT_DIRECTORY = "gs://{}/batch_predictions".format(BUCKET)
MACHINE_TYPE = "n1-standard-2"
STARTING_REPLICA_COUNT = 1
BATCH_SIZE = 64

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/batchPredictionJobs".format(
    REGION, PROJECT, REGION
)
headers = {"Authorization": "Bearer " + token}
data = {
    "displayName": BATCH_JOB_NAME,
    "model": "projects/{}/locations/{}/models/{}".format(
        PROJECT, REGION, MODEL_ID
    ),
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "gcsSource": {
            "uris": [INPUT_URI],
        },
    },
    "outputConfig": {
        "predictionsFormat": "jsonl",
        "gcsDestination": {
            "outputUriPrefix": OUTPUT_DIRECTORY,
        },
    },
    "dedicatedResources" : {
        "machineSpec" : {
            "machineType": MACHINE_TYPE
        },
        "startingReplicaCount": STARTING_REPLICA_COUNT
    },
    "manualBatchTuningParameters": {
        "batch_size": BATCH_SIZE,
    }
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

## [Optional] Invoking online predictions from Apache Beam

In [ ]:
import apache_beam as beam
import json
from oauth2client.client import GoogleCredentials
import requests

class ModelPredict:
    def __init__(self, project, region, endpoint_id):
        self._api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
            region, project, region, endpoint_id)   
        
    def __call__(self, filenames):        
        token = GoogleCredentials.get_application_default().get_access_token().access_token
        if isinstance(filenames, str):
            # Only one element, put it into a batch of 1.
            data = {
                "instances": [
                    {"filenames": filenames}
                ]
            }
        else:
            data = {
                "instances": []
            }
            for f in filenames:
                data["instances"].append({
                    "filenames" : f
                })
        # print(data)
        headers = {"Authorization": "Bearer " + token }
        response = requests.post(self._api, json=data, headers=headers)
        response = json.loads(response.content.decode("utf-8"))
        # print(response)
        if isinstance(filenames, str):
            result = response["predictions"][0]
            result["filename"] = filenames
            yield result
        else:
            for (a,b) in zip(filenames, response["predictions"]):
                result = b
                result["filename"] = a
                yield result


PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

with beam.Pipeline() as p:    
    (p 
     | "input" >> beam.Create([
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg",
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg",
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg",
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg",
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
     ]) 
     | "batch" >> beam.BatchElements(min_batch_size=2, max_batch_size=3)
     | "addpred" >> beam.FlatMap(ModelPredict(PROJECT, REGION, ENDPOINT_ID))
     | "write" >> beam.Map(print)
    )

## License
Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.